In [1]:
import tensorflow as tf
import pandas as pd
import tqdm
import numpy as np
import cv2

In [2]:
df= pd.read_csv('Data/train.csv')
#df = df[:100]
all_id = np.array(df['Id'])

In [3]:
def get_images_from_id(data_id, _type = 'train', img_size=128, colours=3):
    pictures = np.zeros((len(data_id), img_size, img_size, colours))
    for i, single_id in tqdm.tqdm(enumerate(data_id)):
        img = cv2.cvtColor(cv2.imread(f"Data/{_type}/{single_id}.jpg"), cv2.COLOR_BGR2RGB)
        if _type == 'train':
            new_img = cv2.resize(img, (img_size, img_size))
        else:
            new_img = img
        pictures[i] = new_img/255
    return pictures

In [4]:
y = df['Pawpularity']
X_image = get_images_from_id(df['Id'])
X_metadata = df.drop(columns=['Id', 'Pawpularity'])
X_metadata.shape[1]

9912it [01:17, 128.59it/s]


12

In [5]:
tf.random.set_seed(42)
IMG_SIZE = 128


input_image = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3)) # Wejście na zdjęcia 128X128x3
input_metadata = tf.keras.Input(X_metadata.shape[1]) # Wejście na metadane 13 kategorii

image_conv2D_1 = tf.keras.layers.Conv2D(8, 3, activation='relu', padding='same')(input_image)
image_conv2D_2 = tf.keras.layers.Conv2D(8, 3, activation='relu', padding='same')(image_conv2D_1)
image_maxpool2D_1 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_2)

image_conv2D_3 = tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same')(image_maxpool2D_1)
image_conv2D_4 = tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same')(image_conv2D_3)
image_maxpool2D_2 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_4)

image_conv2D_5 = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(image_maxpool2D_2)
image_conv2D_6 = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(image_conv2D_5)
image_maxpool2D_3 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_6)

image_conv2D_7 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(image_maxpool2D_3)
image_conv2D_8 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(image_conv2D_7)
image_maxpool2D_4 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_8)

image_conv2D_9 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(image_maxpool2D_4)
image_conv2D_10 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(image_conv2D_9)
image_maxpool2D_5 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_10) 
flatten = tf.keras.layers.GlobalAveragePooling2D()(image_maxpool2D_5) #Podobne działanie do Flatten

metadata_dense_1 = tf.keras.layers.Dense(16)(input_metadata)
metadata_dense_2 = tf.keras.layers.Dense(16)(metadata_dense_1)
metadata_dense_3 = tf.keras.layers.Dense(16)(metadata_dense_2)
metadata_dense_4 = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer=tf.keras.regularizers.l2())(metadata_dense_3)

concat = tf.keras.layers.concatenate([flatten, metadata_dense_4])
output = tf.keras.layers.Dense(1)(concat)
model = tf.keras.Model(inputs=[input_image, input_metadata], outputs=[output])

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(1e-3), metrics=["mse"])

history_1 = model.fit((X_image, X_metadata), y, epochs=50)

Train on 9912 samples
Epoch 1/50
9912/9912 [==============================] - 13s 1ms/sample - loss: 485.0391 - mse: 484.8879
Epoch 2/50
9912/9912 [==============================] - 9s 905us/sample - loss: 437.2348 - mse: 437.0974
Epoch 3/50
9912/9912 [==============================] - 9s 900us/sample - loss: 433.5548 - mse: 433.4339
Epoch 4/50
9912/9912 [==============================] - 9s 894us/sample - loss: 432.5574 - mse: 432.4505
Epoch 5/50
9912/9912 [==============================] - 9s 896us/sample - loss: 432.1709 - mse: 432.0742
Epoch 6/50
9912/9912 [==============================] - 9s 936us/sample - loss: 429.8434 - mse: 429.7533
Epoch 7/50
9912/9912 [==============================] - 9s 921us/sample - loss: 431.3286 - mse: 431.2431
Epoch 8/50
9912/9912 [==============================] - 9s 922us/sample - loss: 429.9459 - mse: 429.8643
Epoch 9/50
9912/9912 [==============================] - 9s 926us/sample - loss: 431.1661 - mse: 431.0876- ETA: 0s - loss: 431.6137 - mse: 4

In [6]:
df_test = pd.read_csv('Data/test.csv', sep=';')
X_images_test = get_images_from_id(df_test['Id'], 'test')
X_metadata_test = df_test.drop(columns=['Id'])

8it [00:00, 2564.34it/s]


In [7]:
X_images_test[0].shape

(128, 128, 3)

In [8]:
results = model.predict((X_images_test, X_metadata_test))
results

array([[36.748814],
       [35.853542],
       [35.617775],
       [37.330677],
       [36.286415],
       [41.93344 ],
       [32.491848],
       [34.817093]], dtype=float32)

In [9]:
sub = pd.read_csv('Data/sample_submission.csv')
sub['Pawpularity'] = 0.001
sub.at[0, 'Pawpularity'] = 10
sub

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,10.000
1,43a2262d7738e3d420d453815151079e,0.001
2,4e429cead1848a298432a0acad014c9d,0.001
3,80bc3ccafcc51b66303c2c263aa38486,0.001
4,8f49844c382931444e68dffbe20228f4,0.001
5,b03f7041962238a7c9d6537e22f9b017,0.001
6,c978013571258ed6d4637f6e8cc9d6a3,0.001
7,e0de453c1bffc20c22b072b34b54e50f,0.001


In [10]:
for i, res in enumerate(results):
    sub.at[i, 'Pawpularity'] = res
sub

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,36.748814
1,43a2262d7738e3d420d453815151079e,35.853542
2,4e429cead1848a298432a0acad014c9d,35.617775
3,80bc3ccafcc51b66303c2c263aa38486,37.330677
4,8f49844c382931444e68dffbe20228f4,36.286415
5,b03f7041962238a7c9d6537e22f9b017,41.933441
6,c978013571258ed6d4637f6e8cc9d6a3,32.491848
7,e0de453c1bffc20c22b072b34b54e50f,34.817093


In [11]:
sub.to_csv('my_submission.csv', index=False)